# HyDroForM Use Case

This notebook showcases how to use the HyDroForM use case, which integrates hydrological modeling with machine learning techniques for efficient and accurate flood forecasting.

The use case setup utilizes openEO to trigger container processing on OSCAR.

## Table of Contents

TODO

## Importing Required Libraries

## Authenticate with openEO backend

EGI Check-in allows us to reuse the same credentials for OSCAR

## Setting up the workflow

The `run_oscar` openEO process developed in InterTwin is used to trigger the execution of the workflow on OSCAR.

It allows us to set up the workflow by providing the service definitions, OSCAR backend URL and the input parameters.

As a best practice for the greatest compatibility with openEO, the container shall return a STAC collection as output.

This is necessary to be able to load the results back into openEO for further processing or visualization.